# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

%load_ext autoreload
%autoreload 2

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#cf): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours): Number of crew or vessel hours spent at sea
 - [Number of Tows](#tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-costs): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#npv): Project NPV calculator
 - [PySAM-Powered Results](#pysam): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 96.1%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.97,0.92,0.98,0.98,0.98,0.98,0.92,0.93,0.98,0.98,0.98,0.91,0.98,0.98,0.98,0.92,0.97,0.86,0.98,0.98,0.98,0.98,0.97,0.96,0.96,0.82,0.98,0.96,0.98,0.98,0.87,0.96,0.98,0.98,0.98,0.87,0.96,0.97,0.91,0.98,0.97,0.96,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.96,0.98,0.92,0.98,0.94,0.98,0.98,0.91,0.96,0.95,0.98,0.95,0.97,0.96,0.99,0.98,0.98,0.95,0.93,0.98,0.98,0.97,0.98,0.98,0.98,0.96,0.98,0.98,0.97,0.98,0.95


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.96
2004,0.96
2005,0.96
2006,0.96
2007,0.95
2008,0.96
2009,0.96
2010,0.97
2011,0.97


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.97
2,0.96
3,0.96
4,0.96
5,0.96
6,0.96
7,0.96
8,0.96
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.98
     2          0.97
     3          0.97
     4          0.97
     5          0.95
     6          0.95
     7          0.98
     8          0.99
     9          0.98
     10         0.96
     11         0.94
     12         0.94
2004 1          0.95
     2          0.97
     3          0.98
     4          0.98
     5          0.95
     6          0.95
     7          0.95
     8          0.95
     9          0.95
     10         0.97
     11         0.97
     12         0.96

<a id="cf"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.45,0.44,0.47,0.47,0.47,0.47,0.44,0.45,0.46,0.47,0.47,0.43,0.47,0.47,0.47,0.44,0.47,0.41,0.47,0.47,0.47,0.47,0.46,0.46,0.46,0.39,0.47,0.46,0.47,0.46,0.41,0.46,0.47,0.46,0.47,0.42,0.46,0.46,0.44,0.47,0.46,0.46,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.45,0.47,0.44,0.47,0.45,0.47,0.46,0.44,0.45,0.45,0.47,0.46,0.46,0.46,0.47,0.47,0.47,0.46,0.44,0.47,0.47,0.46,0.47,0.47,0.46,0.46,0.47,0.47,0.46,0.47,0.45


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.40
2004,0.45
2005,0.47
2006,0.45
2007,0.48
2008,0.50
2009,0.46
2010,0.41
2011,0.49


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.50
4,0.41
5,0.39
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.36
     3          0.37
     4          0.52
     5          0.33
     6          0.30
     7          0.30
     8          0.30
     9          0.27
     10         0.42
     11         0.53
     12         0.53
2004 1          0.53
     2          0.51
     3          0.51
     4          0.40
     5          0.29
     6          0.37
     7          0.26
     8          0.40
     9          0.56
     10         0.56
     11         0.48
     12         0.48

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.97
2004,0.99
2005,1.02
2006,1.00
2007,1.00
2008,1.00
2009,1.01
2010,1.00
2011,1.00


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.93
2,1.02
3,1.05
4,1.00
5,1.01
6,1.03
7,1.00
8,0.99
9,0.98


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.82
     2                 1.13
     3                 0.94
     4                 1.02
     5                 0.99
     6                 1.02
     7                 1.06
     8                 1.00
     9                 0.95
     10                0.99
     11                0.86
     12                0.97
2004 1                 0.80
     2                 0.77
     3                 1.46
     4                 1.11
     5                 1.02
     6                 1.11
     7                 1.13
     8                 0.96
     9                 0.92
     10                1.04
     11                0.87
     12                0.97

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $405,727.51/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,900,129.20","76,294,692.71"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"8,012,674.46"
2004,"15,128,148.07"
2005,"7,288,871.00"
2006,"1,916,250.00"
2007,"15,173,933.33"
2008,"8,349,937.56"
2009,"11,909,917.96"
2010,"5,237,284.18"
2011,"16,616,411.40"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,083.33","543,083.33","543,083.33","51,779.40","16,581,090.28"
2,"496,052.08","496,052.08","496,052.08","385,565.42","3,610,937.50"
3,"541,625.00","541,625.00","541,989.58","186,043.87","2,540,005.21"
4,"525,000.00","525,000.00","524,635.42","159,468.06","5,786,779.51"
5,"542,500.00","542,500.00","542,500.00","252,285.91","3,055,407.99"
6,"525,000.00","525,000.00","525,000.00","397,593.36","1,424,451.39"
7,"542,500.00","542,500.00","542,500.00","180,500.00","4,411,043.40"
8,"542,500.00","542,500.00","542,500.00",0.00,"4,473,437.50"
9,"525,000.00","525,000.00","525,000.00","82,882.44","8,148,680.56"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00       1,424,451.39  
     7                   54,250.00                  0.00       3,600,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00             74,703.98               0.00  
     12                  54,250.00             26,916.67         968,821.18  
2004 1                   54,250.00                  0.00       4,650,000.00  
     2                   50,750.00                  0.00       1,996,875.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00             82,882.44       4,292,328.12  
     10                  54,250.00             14,250.00       2,170,312.50  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

<a id="utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.91,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.93,0.91
2004,1.00,1.00,1.00,0.85,0.99
2005,1.00,1.00,1.00,0.83,0.97
2006,1.00,1.00,1.00,0.00,0.00
2007,1.00,1.00,1.00,0.94,0.98
2008,1.00,1.00,1.00,0.83,0.92
2009,1.00,1.00,1.00,0.93,0.98
2010,1.00,1.00,1.00,0.85,0.94
2011,1.00,1.00,1.00,0.95,0.97


<a id="vessel-crew-hours"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"123,551.50"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 3,Crew Transfer Vessel 2,Crew Transfer Vessel 1,Heavy Lift Vessel,Field Support Vessel
0,2003,"10,276.34","2,025.50","1,920.75","1,881.25","3,312.84","1,136.00"
1,2004,"15,638.52","3,930.50","3,996.00","4,078.50","2,522.02","1,111.50"
2,2005,"13,428.77","3,489.50","3,472.50","3,381.75","1,981.02","1,104.00"
3,2006,"8,646.75","2,912.25","2,894.25","2,840.25",0.00,0.00
4,2007,"14,276.50","3,139.50","3,249.00","3,099.00","4,188.00",601.00
5,2008,"13,680.75","3,207.25","3,087.00","2,864.75","3,417.75","1,104.00"
6,2009,"11,482.79","2,814.50","2,727.75","2,939.50","2,379.04",622.00
7,2010,"11,156.75","2,909.00","2,753.25","2,653.75","1,736.75","1,104.00"
8,2011,"13,578.22","2,631.25","2,657.75","2,977.00","4,044.72","1,267.50"
9,2012,"11,386.10","2,775.50","2,434.50","2,489.00","2,008.10","1,679.00"


<a id="tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

<a id="equipment-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,468,447.92"
Repair,"19,265,962.67"
Crew Transfer,"1,131,244.80"
Site Travel,0.00
Mobilization,"6,500,000.00"
Weather Delay,"38,438,456.60"
No Requests,"8,989,863.07"
Not in Shift,"20,580,628.10"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,468,447.92","2,468,447.92"
Repair,0.00,0.00,0.00,"19,265,962.67","19,265,962.67"
Crew Transfer,0.00,0.00,0.00,"1,131,244.80","1,131,244.80"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"6,500,000.00","6,500,000.00"
Weather Delay,0.00,0.00,0.00,"38,438,456.60","38,438,456.60"
No Requests,0.00,0.00,0.00,"8,989,863.07","8,989,863.07"
Not in Shift,0.00,0.00,0.00,"20,580,628.10","20,580,628.10"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      75,414.06
     Repair        1,631,219.62
     Crew Transfer    91,348.96
     Site Travel           0.00
     Mobilization  1,000,000.00
     Weather Delay 2,047,500.00
     No Requests   1,486,576.39
     Not in Shift  1,680,615.43
2004 Maintenance     431,867.19
     Repair        2,437,924.48

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                     
1     Maintenance      149,843.75
      Repair         3,129,462.67
      Crew Transfer    156,002.60
      Site Travel            0.00
      Mobilization     500,000.00
      Weather Delay 10,467,354.17
      No Requests      684,377.60
      Not in Shift   3,175,078.88
2     Maintenance      161,656.25
      Repair           813,104.17

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,161.46
           Crew Transfer    2,296.88
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   19,760.42
           No Requests    114,296.88
           Not in Shift     7,765.62
     2     Maintenance          0.00
           Repair          25,575.52
           Crew Transfer    3,098.96
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   13,781.25
           No Requests     92,111.98
           Not in Shift    12,432.29
     3     Maintenance          0.00
           Repair          26,614.58
           Crew Transfer    2,734.38
           Site Travel          0.00

<a id="component-costs"></a>
## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"117,954,790.26"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   58,851,634.87   
          maintenance            0.00              0.00    2,468,447.92   
          repair                 0.00              0.00   19,265,962.67   

                         total_cost  
component action                     
turbine   delay       58,851,634.87  
          maintenance  2,468,447.92  
          repair      19,265,962.67

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"2,516,000.00",0.00,"5,510,237.85","8,026,237.85"
2004,turbine,"5,136,500.00",0.00,"14,461,083.33","19,597,583.33"
2005,turbine,"3,885,500.00",0.00,"6,369,061.63","10,254,561.63"
2006,turbine,"1,689,500.00",0.00,"1,157,479.17","2,846,979.17"
2007,turbine,"4,452,500.00",0.00,"13,333,298.61","17,785,798.61"
2008,turbine,"3,188,000.00",0.00,"6,334,585.07","9,522,585.07"
2009,turbine,"4,557,500.00",0.00,"10,396,054.69","14,953,554.69"
2010,turbine,"2,431,000.00",0.00,"3,894,919.60","6,325,919.60"
2011,turbine,"5,748,500.00",0.00,"14,161,836.81","19,910,336.81"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"4,539,000.00",0.00,"17,061,431.42","21,600,431.42"
2,turbine,"2,348,500.00",0.00,"4,793,455.73","7,141,955.73"
3,turbine,"2,125,000.00",0.00,"3,237,502.61","5,362,502.61"
4,turbine,"3,451,500.00",0.00,"6,104,573.09","9,556,073.09"
5,turbine,"2,667,500.00",0.00,"3,973,043.55","6,640,543.55"
6,turbine,"1,904,500.00",0.00,"1,971,136.29","3,875,636.29"
7,turbine,"2,674,000.00",0.00,"4,485,584.20","7,159,584.20"
8,turbine,"3,271,000.00",0.00,"5,503,986.98","8,774,986.98"
9,turbine,"2,845,000.00",0.00,"7,699,696.18","10,544,696.18"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         36,500.00              0.00       48,453.13   
     2     turbine         41,500.00              0.00       53,138.02   
     3     turbine        108,500.00              0.00       45,427.08   
     4     turbine         37,500.00              0.00       41,617.19   
     5     turbine         59,000.00              0.00       37,296.88   
     6     turbine         32,500.00              0.00      946,326.39   
     7     turbine      1,408,500.00              0.00    3,341,303.82   
     8     turbine         61,000.00              0.00       52,481.77   
     9     turbine         51,000.00              0.00       41,434.90   
     10    turbine         73,500.00              0.00       74,575.52   
     11    turbine        351,500.00              0.00      176,195.31   
     12    turbine        255,000.00              0.00      651,987.85   
2004 1     turbine      1,212,500.00              0.00    4,812,750.00   
     2     turbine        461,500.00              0.00    2,147,375.00   
     3     turbine        279,000.00              0.00      161,875.00   
     4     turbine        261,500.00              0.00      154,875.00   
     5     turbine        296,500.00              0.00      161,875.00   
     6     turbine        219,500.00              0.00      156,625.00   
     7     turbine        259,500.00              0.00      139,143.23   
     8     turbine        108,500.00              0.00       71,950.52   
     9     turbine        729,000.00              0.00    4,008,593.75   
     10    turbine        904,000.00              0.00    2,326,645.83   
     11    turbine        217,500.00              0.00      157,500.00   
     12    turbine        187,500.00              0.00      161,875.00   
2005 1     turbine        150,500.00              0.00      159,250.00   
     2     turbine        131,000.00              0.00      141,750.00   
     3     turbine        188,500.00              0.00      158,265.63   
     4     turbine        131,000.00              0.00      115,755.21   

                       total_cost  
year month component               
2003 1     turbine      84,953.13  
     2     turbine      94,638.02  
     3     turbine     153,927.08  
     4     turbine      79,117.19  
     5     turbine      96,296.88  
     6     turbine     978,826.39  
     7     turbine   4,749,803.82  
     8     turbine     113,481.77  
     9     turbine      92,434.90  
     10    turbine     148,075.52  
     11    turbine     527,695.31  
     12    turbine     906,987.85  
2004 1     turbine   6,025,250.00  
     2     turbine   2,608,875.00  
     3     turbine     440,875.00  
     4     turbine     416,375.00  
     5     turbine     458,375.00  
     6     turbine     376,125.00  
     7     turbine     398,643.23  
     8     turbine     180,450.52  
     9     turbine   4,737,593.75  
     10    turbine   3,230,645.83  
     11    turbine     375,000.00  
     12    turbine     349,375.00  
2005 1     turbine     309,750.00  
     2     turbine     272,750.00  
     3     turbine     346,765.63  
     4     turbine     246,755.21

<a id="fixed-costs"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [49]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [50]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [51]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [52]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [53]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [54]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [55]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low").head(28)

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
     6      131,507.51
     7      135,891.09
     8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09
2004 1      135,891.09
     2      127,123.92
     3      135,891.09
     4      131,507.51
     5      135,891.09
     6      131,507.51
     7      135,891.09
     8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09
2005 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [56]:
metrics.opex("project")

,OpEx
0,"149,625,333.91"


In [57]:
metrics.opex("annual")

,OpEx
year,
2003,"12,128,682.46"
2004,"21,869,039.65"
2005,"12,774,379.00"
2006,"5,205,758.00"
2007,"21,226,441.33"
2008,"13,142,329.14"
2009,"18,067,425.96"
2010,"9,268,292.18"
2011,"23,964,919.40"


In [58]:
metrics.opex("monthly")

,OpEx
month,
1,"24,160,030.58"
2,"9,073,713.32"
3,"7,835,199.56"
4,"12,287,458.06"
5,"8,961,604.81"
6,"6,616,619.82"
7,"10,251,954.31"
8,"10,730,848.40"
9,"13,966,638.07"


In [59]:
metrics.opex("month-year").head(28)

OpEx
year month             
2003 1       336,672.34
     2       311,240.34
     3       407,141.09
     4       326,507.51
     5       357,641.09
     6     1,745,958.90
     7     5,307,141.09
     8       359,641.09
     9       340,007.51
     10      372,141.09
     11      715,211.48
     12    1,549,378.94
2004 1     6,161,141.09
     2     2,737,748.92
     3       577,641.09
     4       550,507.51
     5       595,141.09
     6       508,507.51
     7       558,141.09
     8       407,141.09
     9     5,393,218.07
     10    3,387,203.59
     11      506,507.51
     12      486,141.09
2005 1       449,141.09
     2       400,740.34
     3       487,141.09
     4       420,007.51

In [60]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2004 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2005 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2006 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2007 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2008 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2009 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2010 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2011 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2012 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2013 1              0

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [61]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"240,890.44","81,425.25","72,853.25",568.00
major repair,"110,788.93","2,312.85","1,613.81",36.00
major replacement,"165,810.96","11,028.49","165,810.96",54.00
manual reset,"226,491.31","37,423.75","12,796.00","5,654.00"
medium repair,"13,662.46","10,691.75","8,375.75",213.00
minor repair,"80,285.61","33,299.00","15,375.25","2,288.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [62]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,638,121,726.50"


In [63]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,638,121,726.50","119,663,764.50","114,972,851.00","122,586,150.00","122,739,637.00","123,339,131.50","123,619,323.00","115,448,521.50","117,577,958.00","122,075,850.00","123,282,665.50","123,049,701.00","113,842,651.50","122,901,779.00","123,296,432.50","122,317,682.00","115,908,106.50","122,373,114.00","106,841,377.00","122,391,736.00","122,765,291.50","123,247,422.00","123,482,416.50","121,152,506.50","120,182,559.50","119,730,978.50","103,454,305.50","122,802,717.50","120,330,554.00","122,403,320.50","121,598,541.50","107,198,303.50","119,937,906.50","123,967,432.50","122,237,410.00","122,612,321.50","109,188,292.50","119,892,921.50","120,921,872.00","115,219,537.00","123,294,412.50","121,260,146.00","120,296,555.50","123,290,756.00","123,016,787.50","123,209,492.50","123,153,897.50","123,597,496.50","122,947,668.00","122,529,155.00","119,528,101.50","122,349,738.00","115,478,454.00","123,048,053.00","117,336,083.00","122,995,302.00","121,724,453.50","115,140,056.00","119,080,217.50","119,088,315.00","122,342,850.50","120,168,513.50","121,358,485.50","121,099,204.00","123,508,955.00","122,371,753.00","122,840,007.50","119,960,111.00","117,035,047.50","122,909,763.50","123,732,597.00","121,785,611.50","122,577,808.00","123,129,361.00","121,876,305.00","120,629,634.00","122,534,918.50","123,251,295.00","121,224,102.00","122,436,956.00","118,428,265.00"


In [64]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"850,428,494.50"
2004,"942,142,794.00"
2005,"984,782,372.50"
2006,"942,516,993.50"
2007,"1,015,618,835.50"
2008,"1,050,478,932.00"
2009,"959,630,610.00"
2010,"862,981,843.00"
2011,"1,028,667,255.00"


In [65]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,077,642,085.50"
2,"821,352,502.50"
3,"900,203,148.50"
4,"711,534,614.50"
5,"691,667,867.00"
6,"558,478,545.50"
7,"587,253,313.00"
8,"638,285,138.00"
9,"789,103,293.00"


In [66]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     110,423,109.00
     2      57,821,557.00
     3      65,965,831.50
     4      90,421,693.50
     5      59,588,136.00
     6      51,298,432.50
     7      52,920,752.50
     8      53,632,299.50
     9      47,092,098.00
     10     75,497,473.50
     11     91,240,780.00
     12     94,526,331.50
2004 1      95,373,716.50
     2      85,285,285.50
     3      91,741,655.00
     4      68,339,204.50
     5      51,416,935.00
     6      64,696,855.50
     7      47,104,909.50
     8      71,312,808.50
     9      97,222,062.00
     10     99,734,853.50
     11     83,607,917.50
     12     86,306,591.00

<a id="npv"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [67]:
metrics.npv("project")

,NPV
0,"195,639,410.19"


In [68]:
metrics.opex("annual")

,OpEx
year,
2003,"12,128,682.46"
2004,"21,869,039.65"
2005,"12,774,379.00"
2006,"5,205,758.00"
2007,"21,226,441.33"
2008,"13,142,329.14"
2009,"18,067,425.96"
2010,"9,268,292.18"
2011,"23,964,919.40"


In [69]:
metrics.opex("monthly")

,OpEx
month,
1,"24,160,030.58"
2,"9,073,713.32"
3,"7,835,199.56"
4,"12,287,458.06"
5,"8,961,604.81"
6,"6,616,619.82"
7,"10,251,954.31"
8,"10,730,848.40"
9,"13,966,638.07"


In [76]:
metrics.opex("month-year").head(28)

OpEx
year month             
2003 1       336,672.34
     2       311,240.34
     3       407,141.09
     4       326,507.51
     5       357,641.09
     6     1,745,958.90
     7     5,307,141.09
     8       359,641.09
     9       340,007.51
     10      372,141.09
     11      715,211.48
     12    1,549,378.94
2004 1     6,161,141.09
     2     2,737,748.92
     3       577,641.09
     4       550,507.51
     5       595,141.09
     6       508,507.51
     7       558,141.09
     8       407,141.09
     9     5,393,218.07
     10    3,387,203.59
     11      506,507.51
     12      486,141.09
2005 1       449,141.09
     2       400,740.34
     3       487,141.09
     4       420,007.51

<a id="pysam"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [71]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [72]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [73]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [74]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [75]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
